# Recursion

A function that calls itself until the problem is solved. Will run until a base case is reached.

In [1]:
var sum_of_10 = Foo(10);
Console.WriteLine(sum_of_10);

int Foo(int n) {
    if (n == 1) return 1;
    return n + Foo(n - 1);
}

55


### Return Address | Return Value | Arguments ###

Recursion burrows down until it finds the base case, then traverses back up filling in the missing return values.

```
Foo(5) | 5 + ? | 5    |
Foo(4) | 4 + ? | 4    |
Foo(3) | 3 + ? | 3    |
Foo(2) | 2 + ? | 2    |
Foo(1) |   1   | 1   \|/

Foo(5) |5 +(10)| 5   /|\
Foo(4) | 4 +(6)| 4    |
Foo(3) | 3 +(3)| 3    |
Foo(2) | 2 +(1)| 2    |
Foo(1) |   1 ^ | 1    |
```

Recursion:
- Pre : `n + __`; `Base Case`;
- Recurse : `Foo(n)`
- Post : `return`; BUT can do more in *Post* step than just return.

### Maze Solver

```fsharp
[|
'####E###'
'#      #'
'#S######'
|]
```

Base case would include running off the map, going somewhere you've been, going backwards at start, etc.

Maze Base Cases:
1. It's a wall.
2. Off the map.
3. It's the end.
4. If we've been here before.

In [53]:
char wall = '█';
readonly var maze = new List<string> {
    "█ ██████",
    "█      █",
    "█  █████",
    "█      █",
    "█ ████ █",
    "█ █    █",
    "█ █ ████",
    "███ ████",
};
var start = new Point{X = 0, Y = 1};
var end = new Point {X = 7, Y = 3};
var solver = new Recursion();
var result = solver.SolveMaze(maze, wall, start, end);

public struct Point {
    public int X {get;set;}
    public int Y {get;set;}
}

class Recursion {
    private int[][] _directions =  new int[4][] { 
        new int[] {-1, 0},
        new int[] {1, 0},
        new int[] {0, -1},
        new int[] {0, 1},
    };


    private bool Walk(List<string> maze, char wall, Point current, Point end, bool[][] seen, ref List<Point> path) {
        // 1. Off the map
        if (current.X < 0 || current.X >= maze[0].Length ||
        current.Y < 0 || current.Y >= maze.Count) {
            return false;
        }

        // 2. on a wall
        if (maze[current.X][current.Y] == wall) {
            return false;
        }

        // 3. Reached the end
        if (current.X == end.X && current.Y == end.Y) {
            path.Add(end);
            return true;
        }

        // 4. Been here before 
        if (seen[current.X][current.Y]) {
            return false;
        }
        
        // 3 recurse steps
        // pre
        seen[current.X][current.Y] = true;
        path.Add(current);
        // recurse
        for (int i = 0; i < 4; ++i){
            var x = _directions[i][0];
            var y = _directions[i][1];
            if (Walk(maze, wall, new Point {X = current.X + x, Y = current.Y + y}, end, seen, ref path)) {
                return true;
            };
        }
        // post (Walking didn't return true)
        path.Remove(current);
        return false;
    }

    public List<Point> SolveMaze(List<string> maze, char wall, Point start, Point end) {
        bool[][] seen = new bool[maze[0].Length][];
        List<Point> path = new();

        for (int i = 0; i < maze.Count; ++i) {
            seen.SetValue(Enumerable.Repeat(false, maze[i].Length).ToArray(), i);
        }
        
        Walk(maze, wall, start, end, seen, ref path);
        return path;
    }
}

var flat_maze = string.Concat(maze);
PrintPath(flat_maze, result);

void PrintPath(string maze, List<Point> result){
    // for square mazes
    Console.WriteLine("Path Taken:");
    var pathed_maze_builder = new StringBuilder(maze);
    int size = (int)Math.Sqrt(maze.Length);

    foreach (var p in result){
        pathed_maze_builder.Remove(size * p.X + p.Y, 1).Insert(size * p.X + p.Y, "*");
    }

    var pathed_maze_flat = pathed_maze_builder.ToString();
    ReadOnlySpan<char> m = pathed_maze_flat.AsSpan();
    for (int i = 0; i < size; ++ i) {
        Console.WriteLine(m.Slice(i * size, size).ToString());
    }
}

Path Taken:
█*██████
█*     █
█* █████
█******█
█ ████*█
█ █****█
█ █*████
███*████


Notes
1. Use recursion when there's no distinct end or there's a *branching factor*.
2. Put as much as possible into the Base Case.
